# **Optimizer Model:** Generates models for triangulaton of a grid using spheres and cones.

# Model Variables

**BEFORE RUNNING THE MODEL:** Add a folder to the drive that has the corresponding grid size in the format: Model_Results_X3_Y3_Z3.  Results will not be saved if folder does not exist!

In [1]:
# Grid size
# Use this variable to set the size of the grid of points
x, y, z = [3,3,3]

# Resolution: Number of points per unit
# Setting this allows for more precise models.  Value has to be >= 1
# Increasing this variable by one increases runtime by roughly time^1.6
res = 1

# Iterations of models
# Not a smart shutoff, the more iterations, the more accurate the model.
iterate = 5000

# Enter the radii and heights of the shapes
sphere_radii = [3,4]

# For example of this, see the "Cone Model Example.png"
# Note that the first value of the radius and heights pair together. They don't both get randomly selected.
cone_radii = [3,4]
cone_heights = [3,4]

# "Save All Models? (Y/N)  *This dramatically increases runtime!*
is_save = False

# Multiple Sensor Types? (Y/N)
# Red only requires one sensor to triagulate, Bule requires 3
sensors = "BR"
# Blue only: sensors = "B"
# Red only: sensor = "R"

# Special Grid? (Y/N)
special_grid = False

# Minimum Shapes (S), Minimum Cost (C), Optimal Shapes (X), Optimal Cost (O)?
# Minimum: Shotgun approach, sprays models to approximate the most accurate
# Optimal: "Smart" approach, attempts to place the "Best" shape each time.
minimize = 'C'

# Filepath of this script
drive_path = "/content/drive/Shareddrives/Prime TTT (IL4)/DevOps/Models/Grid Optimizer Model/"

In [2]:
def sphere_cost(factor, type="B"):
    if type == "B":
        return volume_sphere(factor)
    else:
        return volume_sphere(factor) * 3

In [3]:
def cone_cost(factor_1, factor_2, type="B"):
    if type == "B":
        return volume_cone(factor_1, factor_2)
    else:
        return volume_cone(factor_1, factor_2) * 3

**Settings and Variables for Graphing**

In [4]:
# Default Resolution of spheres graphed.  Recommend = 5 for testing, 80 for high quality outputs
RESOLUTION = 10
# Default Resolution of spheres graphed.  Number from 0 to 1
DEFAULT_OPACITY = .3
# Captures the graph-able data
fig_data = []
# Captures the data from the shapes that need graphed
shape_data = []

**Imports**

In [5]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import os
import tkinter.filedialog as fd
import math
import random
from operator import itemgetter
from google.colab import drive
drive.mount('/content/drive')

is_spheres = False
if sphere_radii:
  is_spheres = True

is_cones = False
if cone_radii:
  is_cones = True

Mounted at /content/drive


#Functions

In [6]:
def distance_finder(x1, y1, z1, x2, y2, z2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)

In [7]:
def is_Intersect(x1, y1, z1, r1, x2, y2, z2, r2):
    if distance_finder(x1, y1, z1, x2, y2, z2) > r1 + r2:
        return False
    return True

In [8]:
def plotly_sphere(x, y, z, radius, resolution=RESOLUTION):
    u, v = np.mgrid[0:2 * np.pi:resolution * 2j, 0:np.pi:resolution * 1j]
    X = radius * np.cos(u) * np.sin(v) + x
    Y = radius * np.sin(u) * np.sin(v) + y
    Z = radius * np.cos(v) + z
    return X, Y, Z

In [9]:
def volume_cone(r, h):
    return (2 / 3) * math.pi * r ** 3 + r ** 2 * ((h * math.pi) / 3)

In [10]:
def volume_sphere(r):
    return (4 / 3) * math.pi * r ** 3

In [11]:
def rotate(xy, theta):
    cos_theta, sin_theta = math.cos(theta), math.sin(theta)

    return (
        xy[0] * cos_theta - xy[1] * sin_theta,
        xy[0] * sin_theta + xy[1] * cos_theta
    )

In [12]:
def translate(xy, offset):
    return xy[0] + offset[0], xy[1] + offset[1]

#Classes

**Neural Network Class**

In [13]:
class NN:
    def __init__(self, grid_x, grid_y, grid_z, resolution, stypes="B", sphere_radii=None, cone_radii=None,
                 cone_heights=None, special_grid=False):
        self.g_x = grid_x
        self.g_y = grid_y
        self.g_z = grid_z
        self.res = resolution
        self.Models = []
        self.sphere_radii = sphere_radii
        self.cone_radii = cone_radii
        self.cone_heights = cone_heights
        self.stypes = stypes
        self.special_grid = special_grid

        if sphere_radii is not None:
            if cone_heights is not None:
                self.shape = 'SC'
            else:
                self.shape = 'S'
        else:
            self.shape = 'C'
        return

    # Print all the models stored in the network
    def print(self):
        for i, m in enumerate(self.Models):
            print(f"Model {i}:")
            m.print()

    # Run the network to find the minimum # spheres to triangulate the grid and return the best model
    def run_minimize_nodes(self, iterations, min_shapes=999999, save_all=False):
        ideal_model = None
        for i in range(0, iterations):
            m = Model(self.g_x, self.g_y, self.g_z, self.res, special_grid=self.special_grid)
            c = 0
            goal = False

            while not (goal or (c >= min_shapes and not save_all)):
                c = c + 1

                rand_shape = random.choice(self.shape)
                if rand_shape == 'C':
                    rand_sensor = random.choice(self.stypes)
                    select = random.randrange(0, len(self.cone_radii), step=1)
                    if self.special_grid:
                        m.Cones.append(
                            m.add_cone_restricted(self.cone_radii[select], self.cone_heights[select], type=rand_sensor))
                    else:
                        m.Cones.append(m.add_cone(self.cone_radii[select], self.cone_heights[select], type=rand_sensor))
                    goal = m.goal_last_cone_polar()
                else:
                    rand_sensor = random.choice(self.stypes)
                    radius_select = random.randrange(0, len(self.sphere_radii), step=1)
                    if self.special_grid:
                        m.Spheres.append(m.add_sphere_restricted(self.sphere_radii[radius_select], type=rand_sensor))
                    else:
                        m.Spheres.append(m.add_sphere(self.sphere_radii[radius_select], type=rand_sensor))
                    goal = m.goal_last_sphere()

            print(f"Iteration {i} Number of M Shapes:{m.num_shapes()}")

            if save_all:
                m.save()

            if c < min_shapes:
                min_shapes = c
                print(f"Minimum Shapes: {min_shapes}")
                m.save()
                ideal_model = m

        # Prints the ideal model
        print(f"Spheres: {len(ideal_model.Spheres)}")
        print(f"Cones: {len(ideal_model.Cones)}")
        ideal_model.print()
        ideal_model.graph()

    def run_optimal_nodes(self, iterations, min_shapes=999999, save_all=False):
        ideal_model = None
        for i in range(0, iterations):
            m = Model(self.g_x, self.g_y, self.g_z, self.res, special_grid=self.special_grid)
            c = 0
            goal = False

            while not (goal or (c >= min_shapes and not save_all)):

                c = c + 1

                rand_shape = random.choice(self.shape)
                if rand_shape == 'C':
                    rand_sensor = random.choice(self.stypes)
                    select = random.randrange(0, len(self.cone_radii), step=1)
                    m.Cones.append(
                        m.add_best_cone(self.cone_radii[select], self.cone_heights[select], type=rand_sensor))
                    goal = m.goal_last_cone_polar()
                else:
                    rand_sensor = random.choice(self.stypes)
                    radius_select = random.randrange(0, len(self.sphere_radii), step=1)
                    m.Spheres.append(m.add_best_sphere(self.sphere_radii[radius_select], type=rand_sensor))
                    goal = m.goal_last_sphere()

            print(f"Iteration {i} Number of M Shapes:{m.num_shapes()}")

            if save_all:
                m.save()

            if c < min_shapes:
                min_shapes = c
                print(f"Minimum Shapes: {min_shapes}")
                m.save()
                ideal_model = m

        # Prints the ideal model
        print(f"Spheres: {len(ideal_model.Spheres)}")
        print(f"Cones: {len(ideal_model.Cones)}")
        ideal_model.print()
        ideal_model.graph()

    # Runs the model, aiming to reduce the cost of the system
    def run_minimize_cost(self, iterations, min_cost=99999999999999, save_all=False):
        ideal_model = None
        for i in range(0, iterations):
            m = Model(self.g_x, self.g_y, self.g_z, self.res, special_grid=self.special_grid)
            goal = False
            cost = 0

            while not (goal or (cost >= min_cost and not save_all)):
              rand_shape = random.choice(self.shape)
              if rand_shape == 'C':
                  rand_sensor = random.choice(self.stypes)
                  select = random.randrange(0, len(self.cone_radii), step=1)
                  cost = cost + cone_cost(self.cone_radii[select], self.cone_heights[select], rand_sensor)
                  if special_grid:
                    m.Cones.append(m.add_cone_restricted(self.cone_radii[select], self.cone_heights[select], type=rand_sensor))
                  else:
                    m.Cones.append(m.add_cone(self.cone_radii[select], self.cone_heights[select], type=rand_sensor))
                  goal = m.goal_last_cone_polar()
              elif rand_shape == 'S':
                  rand_sensor = random.choice(self.stypes)
                  radius_select = random.randrange(0, len(self.sphere_radii), step=1)
                  cost = cost + sphere_cost(self.sphere_radii[radius_select], rand_sensor)
                  if special_grid:
                    m.Spheres.append(m.add_sphere_restricted(self.sphere_radii[radius_select], type=rand_sensor))
                  else:
                    m.Spheres.append(m.add_sphere(self.sphere_radii[radius_select], type=rand_sensor))
                  goal = m.goal_last_sphere()

            print(f"Iteration {i} Number of M Shapes:{m.num_shapes()}")

            if save_all:
                m.save()

            if cost < min_cost:
                min_cost = cost
                print(f"Minimum Cost: {min_cost}")
                m.save()
                ideal_model = m

        # Prints the ideal model
        print(f"Spheres: {len(ideal_model.Spheres)}")
        print(f"Cones: {len(ideal_model.Cones)}")
        ideal_model.print()
        ideal_model.graph()
        return

    def run_optimal_cost(self, iterations, min_cost=99999999999999, save_all=False):
        ideal_model = None
        for i in range(0, iterations):
            m = Model(self.g_x, self.g_y, self.g_z, self.res, special_grid=self.special_grid)
            goal = False
            cost = 0

            while not (goal or (cost >= min_cost and not save_all)):
                rand_shape = random.choice(self.shape)

                if rand_shape == 'C':
                    rand_sensor = random.choice(self.stypes)
                    select = random.randrange(0, len(self.cone_radii), step=1)
                    cost = cost + cone_cost(self.cone_radii[select], self.cone_heights[select], rand_sensor)
                    m.Cones.append(
                        m.add_best_cone(self.cone_radii[select], self.cone_heights[select], type=rand_sensor))
                    goal = m.goal_last_cone_polar()

                elif rand_shape == 'S':
                    rand_sensor = random.choice(self.stypes)
                    radius_select = random.randrange(0, len(self.sphere_radii), step=1)
                    cost = cost + sphere_cost(self.sphere_radii[radius_select], rand_sensor)
                    m.Spheres.append(m.add_best_sphere(self.sphere_radii[radius_select], type=rand_sensor))
                    goal = m.goal_last_sphere()

            print(f"Iteration {i} Number of M Shapes:{m.num_shapes()}")

            if save_all:
                m.save()

            if cost < min_cost:
                min_cost = cost
                print(f"Minimum Cost: {min_cost}")
                m.save()
                ideal_model = m

        # Prints the ideal model
        print(f"Spheres: {len(ideal_model.Spheres)}")
        print(f"Cones: {len(ideal_model.Cones)}")
        ideal_model.print()
        ideal_model.graph()
        return

**Instance of Model**

In [14]:
class Model:
    def __init__(self, grid_x, grid_y, grid_z, grid_res, stypes="B", special_grid=False):
        self.Spheres = []
        self.Cones = []
        self.Points = []
        self.res = grid_res
        self.stypes = stypes
        self.special_grid = special_grid
        if not special_grid:
            self.x = grid_x
            self.y = grid_y
            self.z = grid_z

            self.Rects = [
                Rect(start_x=0, len_x=self.x, start_y=0, len_y=self.y, start_z=0, len_z=self.z, rotate_degrees=0)
            ]

        else:
            self.Rects = [
                Rect(start_x=0, len_x=30, start_y=86, len_y=200, start_z=0, len_z=30, rotate_degrees=0),
                Rect(start_x=0, len_x=30, start_y=286, len_y=100, start_z=0, len_z=30, rotate_degrees=30),
                Rect(start_x=0, len_x=30, start_y=0, len_y=100, start_z=0, len_z=30, rotate_degrees=-30)
            ]

            self.x = 90
            self.y = 400
            self.z = 30

        self.grid_generation()
        return

    def grid_generation_rotate_rect(self, degree, len_x, len_y, len_z, y_start=0, x_start=0, z_start=0, rotate="C",
                                    res=1):
        a = math.cos(math.radians(degree))
        b = math.sin(math.radians(degree))

        if rotate == "C":
            for x in np.linspace(0, len_x, (res * len_x) + 1):
                for y in np.linspace(0, len_y, (res * len_y) + 1):
                    y_actual = y_start + y * b + (len_x - x) * a
                    x_actual = x_start + y * a
                    for z in np.linspace(0, len_z, (res * len_z) + 1):
                        self.Points.append([round(x_actual, 3), round(y_actual, 3), z + z_start, 0])
        else:
            for x in np.linspace(0, len_x, (res * len_x) + 1):
                for y in np.linspace(0, len_y, (res * len_y) + 1):
                    y_actual = y_start + (len_y - y) * b + x * a
                    x_actual = x_start + (len_x - x) + y * a
                    for z in np.linspace(0, len_z, (res * len_z) + 1):
                        self.Points.append([round(x_actual, 3), round(y_actual), z + z_start, 0])

    def grid_generation(self):
        data = []
        for rect in self.Rects:
          if rect.deg == 0:
            self.grid_generation_rect(grid_x=rect.l_x, grid_y=rect.l_y, grid_z=rect.l_z, start_x=rect.s_x,
                                          start_y=rect.s_y, start_z=rect.s_x, res=self.res)
          if rect.deg > 0:
            self.grid_generation_rotate_rect(degree=rect.deg, len_x=rect.l_x, len_y=rect.l_y, len_z=rect.l_z,
                                                 x_start=rect.s_x, y_start=rect.s_y, z_start=rect.s_z, rotate="C",
                                                 res=self.res)

          if rect.deg < 0:
            self.grid_generation_rotate_rect(degree=abs(rect.deg), len_x=rect.l_x, len_y=rect.l_y, len_z=rect.l_z,
                                                 x_start=rect.s_x, y_start=rect.s_y, z_start=rect.s_z, rotate="CC",
                                                 res=self.res)
          data.append(rect.graph())
        return

    def grid_generation_rect(self, grid_x, grid_y, grid_z, res=1, start_x=0, start_y=0, start_z=0):
        for x in np.linspace(0, grid_x, (res * grid_x) + 1):
            for y in np.linspace(0, grid_y, (res * grid_y) + 1):
                for z in np.linspace(0, grid_z, (res * grid_z) + 1):
                    self.Points.append([x + start_x, y + start_y, z + start_z, 0])
        return

    # Saves the Model's Spheres and Parameters
    def save(self):
        total_cost = 0
        for sphere in self.Spheres:
            total_cost = total_cost + sphere_cost(sphere.r)
        for cone in self.Cones:
            total_cost = total_cost + cone_cost(cone.r, cone.h)

        data = []
        data_headers = ["X","Y","Z","Radius","Height","Cost","Volume","Shape","Type","X2","Y2","Z2","Degrees"]

        for sphere in self.Spheres:
          data.append(sphere.save())
        for cone in self.Cones:
          data.append(cone.save())

        df = pd.DataFrame(columns=data_headers,data=data)
        df.to_csv(drive_path + f"Model_Save_X{self.x}_Y{self.y}_Z{self.z}/Model_Save_X{self.x}_Y{self.y}_Z{self.z}_Res{self.res}_Cost_{round(sum(df['Cost']))}.csv")
        return

    # Prints the model's Spheres
    def print(self):
        for i, s in enumerate(self.Spheres):
            print(f"Sphere {i}:")
            s.print()
        for i, c in enumerate(self.Cones):
            print(f"Cone {i}:")
            c.print()

    # Returns the number of Spheres in a model
    def num_shapes(self):
        return len(self.Spheres) + len(self.Cones)

    # Checks the last sphere if it overlaps any of the grid's points, and updates accordingly
    def goal_last_sphere(self):
        goal = True
        for point in self.Points:
            if point[3] < 3:
              if distance_finder(self.Spheres[-1].x, self.Spheres[-1].y, self.Spheres[-1].z, point[0], point[1], point[2]) <= self.Spheres[-1].r:
                if self.Spheres[-1].t == "B":
                  point[3] = point[3] + 1
                else:
                  point[3] = point[3] + 3
                if point[3] < 3:
                  goal = False
              else:
                if point[3] < 3:
                  goal = False
        return goal

    def goal_last_cone_polar(self):
        # Initialize the Cone
        O = np.array((self.Cones[-1].x, self.Cones[-1].y, self.Cones[-1].z))  # coordinate of the origin
        T = np.array((self.Cones[-1].tip_x, self.Cones[-1].tip_y, self.Cones[-1].tip_z))  # coordinate of the tip

        # Find the height of the cone
        H = np.linalg.norm(O - T)

        # Define a unit vector that points from T to O
        h = (O - T) / H

        # Find the "side" of the cone
        S = math.sqrt(H ** 2 + self.Cones[-1].r ** 2)

        # Find the half cone angle
        alpha = math.atan2(self.Cones[-1].r, H)

        goal = True

        for point in self.Points:
            if point[3] < 3:
                P = np.array((point[0], point[1], point[2]))

                # In order for P be inside the cone, it must satisfy the following two conditions:
                # 1. |TP| <= S
                # 2. Angle between <TP> and <TO> <= alpha

                # Check condition 1
                TP = np.linalg.norm(P - T)
                if TP <= S:
                    # Check condition 2
                    if math.acos(round(np.dot((P - T), h) / TP, 6)) <= alpha:
                        if self.Cones[-1].t == "B":
                            point[3] = point[3] + 1
                        else:
                            point[3] = point[3] + 3

                if point[3] < 3:
                    goal = False
        return goal

    # Graph the model
    def graph(self):
        data = []

        # Spheres graphing
        for sphere in self.Spheres:
            data.append(sphere.graph())

        # Cone graphing
        for cone in self.Cones:
            data.append(cone.graph_cone())
            data.append(cone.graph_sphere())

        # Grid of coverage defined by initial parameters
        for rect in self.Rects:
            data.append(rect.graph())


        fig = go.Figure(data=data)

        fig.update_traces(showscale=False)
        fig.update_layout(
          showlegend=False,
          autosize=False,
          width=1500,
          height=1000,
          paper_bgcolor="LightSteelBlue",
        )
        fig.show()
        return

    # Add a cone with a radius to the model
    def add_cone(self, radius, height, type="B"):
        is_repeat = True
        x_seed = 0
        y_seed = 0
        z_seed = 0

        r_seed = int(radius / 2) * self.res

        counter = 0

        while is_repeat and counter < 1000:
            counter = counter + 1
            x_seed = (random.randrange(start=0, stop=(self.x + r_seed) * self.res, step=1)) / self.res
            y_seed = (random.randrange(start=0, stop=(self.y + r_seed) * self.res, step=1)) / self.res
            z_seed = (random.randrange(start=0, stop=(self.z + r_seed) * self.res, step=1)) / self.res

            if self.num_shapes() == 0:
                is_repeat = False

            for sphere in self.Spheres:
                if sphere.x != x_seed and sphere.y != y_seed and sphere.z != z_seed:
                    is_repeat = False

            for cone in self.Cones:
                if cone.x != x_seed and cone.y != y_seed and cone.z != z_seed:
                    is_repeat = False

        (X, Y, Z) = plotly_sphere(x_seed, y_seed, z_seed, height)

        select = random.randrange(0, len(X), step=1)
        select_2 = random.randrange(0, len(X[select]), step=1)

        return Cone(x_seed, y_seed, z_seed, radius, height, X[select][select_2], Y[select][select_2],
                    Z[select][select_2], type=type)

    # Add a sphere with a radius to the model
    def add_sphere(self, radius, type="B"):
        is_repeat = True
        x_seed = 0
        y_seed = 0
        z_seed = 0

        r_seed = int(radius / 2) * self.res

        counter = 0

        while is_repeat and counter < 1000:
            counter = counter + 1
            x_seed = (random.randrange(start=0 - r_seed, stop=(self.x + r_seed) * self.res, step=1)) / self.res
            y_seed = (random.randrange(start=0 - r_seed, stop=(self.y + r_seed) * self.res, step=1)) / self.res
            z_seed = (random.randrange(start=0 - r_seed, stop=(self.z + r_seed) * self.res, step=1)) / self.res

            if self.num_shapes() == 0:
                is_repeat = False

            for sphere in self.Spheres:
                if sphere.x != x_seed and sphere.y != y_seed and sphere.z != z_seed:
                    is_repeat = False

            for cone in self.Cones:
                if cone.x != x_seed and cone.y != y_seed and cone.z != z_seed:
                    is_repeat = False

        return Sphere(x_seed, y_seed, z_seed, radius, type=type)

    def add_cone_restricted(self, radius, height, type="B"):
        is_repeat = True
        x_seed = 0
        y_seed = 0
        z_seed = 0

        while is_repeat:
            y_seed = random.randrange(start=0, stop=self.y, step=1) / self.res
            z_seed = random.randrange(start=0, stop=self.z, step=1) / self.res

            if y_seed > 100:
                if y_seed > 300:
                    x_seed = random.randrange(start=int(15 + math.tan(math.radians(30)) * (y_seed - 300)), stop=self.x,
                                              step=1) / self.res
                else:
                    x_seed = random.randrange(start=15, stop=self.x, step=1) / self.res
            else:
                x_seed = random.randrange(start=int(15 + math.tan(math.radians(30)) * (100 - y_seed)), stop=self.x,
                                          step=1) / self.res

            if self.num_shapes() == 0:
                is_repeat = False

            for sphere in self.Spheres:
                if sphere.x != x_seed and sphere.y != y_seed and sphere.z != z_seed:
                    is_repeat = False

            for cone in self.Cones:
                if cone.x != x_seed and cone.y != y_seed and cone.z != z_seed:
                    is_repeat = False

        (X, Y, Z) = plotly_sphere(x_seed, y_seed, z_seed, height)

        select = random.randrange(0, len(X), step=1)
        select_2 = random.randrange(0, len(X[select]), step=1)

        return Cone(x_seed, y_seed, z_seed, radius, height, X[select][select_2], Y[select][select_2],
                    Z[select][select_2], type=type)

    def add_sphere_restricted(self, radius, type="B"):
        is_repeat = True
        x_seed = 0
        y_seed = 0
        z_seed = 0

        while is_repeat:
            y_seed = random.randrange(start=0, stop=self.y, step=1) / self.res
            z_seed = random.randrange(start=0, stop=self.z, step=1) / self.res

            if y_seed > 100:
                if y_seed > 300:

                    x_seed = random.randrange(start=int(15 + math.tan(math.radians(30)) * (y_seed - 300)), stop=self.x,
                                              step=1) / self.res
                else:
                    x_seed = random.randrange(start=15, stop=self.x, step=1) / self.res
            else:

                x_seed = random.randrange(start=int(15 + math.tan(math.radians(30)) * (100 - y_seed)), stop=self.x,
                                          step=1) / self.res

            if self.num_shapes() == 0:
                is_repeat = False

            for sphere in self.Spheres:
                if sphere.x != x_seed and sphere.y != y_seed and sphere.z != z_seed:
                    is_repeat = False

            for cone in self.Cones:
                if cone.x != x_seed and cone.y != y_seed and cone.z != z_seed:
                    is_repeat = False

        return Sphere(x_seed, y_seed, z_seed, radius, type=type)

    def add_best_cone(self, radius, height, type="B", iterations=500):
        best_cone = []
        max_num_points = 0

        for i in range(0, iterations):
            cone = self.add_cone(radius, height)

            num_points = 0

            # Initialize the Cone
            O = np.array((cone.x, cone.y, cone.z))  # coordinate of the origin
            T = np.array((cone.tip_x, cone.tip_y, cone.tip_z))  # coordinate of the tip

            # Find the height of the cone
            H = np.linalg.norm(O - T)

            # Define a unit vector that points from T to O
            h = (O - T) / H

            # Find the "side" of the cone
            S = math.sqrt(H ** 2 + cone.r ** 2)

            # Find the half cone angle
            alpha = math.atan2(cone.r, H)

            for point in self.Points:
                if point[3] < 3:
                    P = np.array((point[0], point[1], point[3]))

                    # In order for P be inside the cone, it must satisfy the following two conditions:
                    # 1. |TP| <= S
                    # 2. Angle between <TP> and <TO> <= alpha

                    # Check condition 1
                    TP = np.linalg.norm(P - T)

                    if TP <= S:
                        # Check condition 2
                        if math.acos(round(np.dot((P - T), h) / TP, 6)) <= alpha:
                            num_points = num_points + 1

            if num_points > max_num_points:
                best_cone = Cone(cone.x, cone.y, cone.z, radius, height, cone.tip_x, cone.tip_y, cone.tip_z, type=type)
        return best_cone

    def add_best_sphere(self, radius, type="B", iterations=500):
        best_sphere = Sphere
        max_num_points = 0

        for i in range(0, iterations):
            sphere = self.add_sphere(radius)

            counter = 0
            for point in self.Points:
                if point[3] < 3:
                    if distance_finder(sphere.x, sphere.y, sphere.z, point[0], point[1],
                                       point[2]) <= sphere.r:
                        if point[3] < 3:
                            counter = counter + 1

            if counter > max_num_points:
                max_num_points = counter
                best_sphere = Sphere(sphere.x, sphere.y, sphere.z, sphere.r, type=type)
        return best_sphere

**Class of Sphere**

In [15]:
class Sphere:
    def __init__(self, origin_x, origin_y, origin_z, radius, type="B"):
        self.x = origin_x
        self.y = origin_y
        self.z = origin_z
        self.r = radius
        self.t = type

    def print(self):
        print(f"X: {self.x}, Y: {self.y}, Z: {self.z}, Radius: {self.r}")

    def graph(self):
      (X, Y, Z) = plotly_sphere(self.x, self.y, self.z, self.r)
      return go.Surface(x=X, y=Y, z=Z, opacity=DEFAULT_OPACITY, colorscale=['green', 'green'])

    def save(self):
      return [self.x, self.y, self.z, self.r, None, sphere_cost(self.r), volume_sphere(self.r), "Sphere", self.t,
                None, None, None,None]

**Class of Cone**

In [16]:
class Cone:
    def __init__(self, origin_x, origin_y, origin_z, radius, height, tip_x, tip_y, tip_z, type="B"):
        self.x = origin_x
        self.y = origin_y
        self.z = origin_z
        self.r = radius
        self.h = height
        self.tip_x = tip_x
        self.tip_y = tip_y
        self.tip_z = tip_z
        self.t = type

    def print(self):
        print(
            f"X: {self.x}, Y: {self.y}, Z: {self.z}, Radius: {self.r}, Height: {self.h}, Tip X: {self.tip_x}, Tip Y: {self.tip_y}, Tip Z: {self.tip_z}")

    def graph_sphere(self):
        (X, Y, Z) = plotly_sphere(self.x, self.y, self.z, self.r)
        return go.Surface(x=X, y=Y, z=Z, opacity=DEFAULT_OPACITY, colorscale=['green', 'green'])

    def graph_cone(self):
        return go.Cone(x=[self.x], y=[self.y], z=[self.z], u=[self.tip_x - self.x], v=[self.tip_y - self.x],
                       w=[self.tip_z - self.z], anchor="tail",
                       opacity=DEFAULT_OPACITY, colorscale=['green', 'green'])

    def save(self):
      return [self.x, self.y, self.z, self.r, self.h, cone_cost(self.r, self.h), volume_cone(self.r, self.h), "Cone",
                self.t, self.tip_x, self.tip_y, self.tip_z, None]


**Class of Rectangle**

In [17]:
class Rect:
    def __init__(self, start_x, len_x, start_y, len_y, start_z, len_z, rotate_degrees):
        self.s_x = start_x
        self.l_x = len_x
        self.s_y = start_y
        self.l_y = len_y
        self.s_z = start_z
        self.l_z = len_z
        self.deg = rotate_degrees
        self.t = "B"
        return

    def print(self):
        print(
            f"X: {self.l_x}, Start X: {self.s_x}, Y: {self.l_y}, Start Y: {self.s_y}, Z: {self.l_z}, Start Z: {self.s_z}, Degrees: {self.deg}")

    def graph(self):
        if self.deg == 0:
            x1 = self.s_x
            x2 = self.l_x + self.s_x
            y1 = self.s_y
            y2 = self.l_y + self.s_y
            z1 = self.s_z
            z2 = self.l_z + self.s_z
            return go.Mesh3d(x=[x1, x1, x2, x2, x1, x1, x2, x2],
                             y=[y1, y2, y2, y1, y1, y2, y2, y1],
                             z=[z1, z1, z1, z1, z2, z2, z2, z2],
                             i=[7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                             j=[3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                             k=[0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                             opacity=1, color='red')
        points = [
            (0, 0),
            (0, self.l_y),
            (self.l_x, self.l_y),
            (self.l_x, 0)
        ]

        rotated_points = []
        if self.deg > 0:
            for xy in points:
                rotated_points.append(translate(rotate(xy, math.radians(-1 * self.deg)), (self.s_x, self.s_y)))

        else:
            for xy in points:
                rotated_points.append(translate(rotate(xy, math.radians(-1 * self.deg)), (
                    round(self.s_x + math.sin(math.radians(-1 * self.deg)) * self.l_y), self.s_y)))

        z1 = self.s_z
        z2 = self.l_z + self.s_z
        return go.Mesh3d(x=[rotated_points[0][0], rotated_points[1][0], rotated_points[2][0], rotated_points[3][0],
                            rotated_points[0][0], rotated_points[1][0], rotated_points[2][0], rotated_points[3][0]],
                         y=[rotated_points[0][1], rotated_points[1][1], rotated_points[2][1], rotated_points[3][1],
                            rotated_points[0][1], rotated_points[1][1], rotated_points[2][1], rotated_points[3][1]],
                         z=[z1, z1, z1, z1, z2, z2, z2, z2],
                         i=[7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                         j=[3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                         k=[0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                         opacity=1, color='red')

#Run Model

In [18]:
if minimize.upper() == 'S':
    if is_spheres and is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors, special_grid=special_grid)
    elif is_spheres:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, stypes=sensors,
                special_grid=special_grid)
    elif is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors, special_grid=special_grid)
    else:
        nn = None
        exit()

    nn.run_minimize_nodes(iterations=iterate, save_all=is_save)

elif minimize.upper() == "C":
    if is_spheres and is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors, special_grid=special_grid)
    elif is_spheres:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, stypes=sensors,
                special_grid=special_grid)
    elif is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors, special_grid=special_grid)
    else:
        nn = None
        exit()
    nn.run_minimize_cost(iterations=iterate, save_all=is_save)

elif minimize.upper() == "X":
    if is_spheres and is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors)
    elif is_spheres:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, stypes=sensors)
    elif is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors)
    else:
        nn = None
        exit()

    nn.run_optimal_nodes(iterations=iterate, save_all=is_save)

else:
    if is_spheres and is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors)
    elif is_spheres:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, sphere_radii=sphere_radii, stypes=sensors)
    elif is_cones:
        nn = NN(grid_x=x, grid_y=y, grid_z=z, resolution=res, cone_radii=cone_radii,
                cone_heights=cone_heights, stypes=sensors)
    else:
        nn = None
        exit()

    nn.run_optimal_cost(iterations=iterate, save_all=is_save)

Iteration 0 Number of M Shapes:16
Minimum Cost: 5378.406622945726
Iteration 1 Number of M Shapes:15
Minimum Cost: 5262.167694762903
Iteration 2 Number of M Shapes:17
Iteration 3 Number of M Shapes:15
Minimum Cost: 5232.846163329398
Iteration 4 Number of M Shapes:17
Minimum Cost: 4882.034983678538
Iteration 5 Number of M Shapes:9
Minimum Cost: 3148.923036448169
Iteration 6 Number of M Shapes:12
Iteration 7 Number of M Shapes:7
Iteration 8 Number of M Shapes:10
Iteration 9 Number of M Shapes:7
Minimum Cost: 2149.896572606615
Iteration 10 Number of M Shapes:9
Iteration 11 Number of M Shapes:7
Iteration 12 Number of M Shapes:8
Iteration 13 Number of M Shapes:7
Iteration 14 Number of M Shapes:6
Iteration 15 Number of M Shapes:6
Iteration 16 Number of M Shapes:8
Iteration 17 Number of M Shapes:8
Iteration 18 Number of M Shapes:3
Minimum Cost: 1520.5308443374597
Iteration 19 Number of M Shapes:4
Iteration 20 Number of M Shapes:6
Iteration 21 Number of M Shapes:7
Iteration 22 Number of M Shape

<ipython-input-14-467e4b97665c>:158: RuntimeWarning: invalid value encountered in scalar divide
  if math.acos(round(np.dot((P - T), h) / TP, 6)) <= alpha:


Iteration 168 Number of M Shapes:4
Iteration 169 Number of M Shapes:2
Iteration 170 Number of M Shapes:5
Iteration 171 Number of M Shapes:2
Iteration 172 Number of M Shapes:5
Iteration 173 Number of M Shapes:3
Iteration 174 Number of M Shapes:2
Iteration 175 Number of M Shapes:4
Iteration 176 Number of M Shapes:2
Iteration 177 Number of M Shapes:4
Iteration 178 Number of M Shapes:4
Iteration 179 Number of M Shapes:3
Iteration 180 Number of M Shapes:2
Iteration 181 Number of M Shapes:4
Iteration 182 Number of M Shapes:3
Iteration 183 Number of M Shapes:3
Iteration 184 Number of M Shapes:2
Iteration 185 Number of M Shapes:3
Iteration 186 Number of M Shapes:5
Iteration 187 Number of M Shapes:2
Iteration 188 Number of M Shapes:4
Iteration 189 Number of M Shapes:4
Iteration 190 Number of M Shapes:5
Iteration 191 Number of M Shapes:4
Iteration 192 Number of M Shapes:2
Iteration 193 Number of M Shapes:2
Iteration 194 Number of M Shapes:2
Iteration 195 Number of M Shapes:4
Iteration 196 Number

<ipython-input-14-467e4b97665c>:158: RuntimeWarning: invalid value encountered in scalar divide
  if math.acos(round(np.dot((P - T), h) / TP, 6)) <= alpha:


Iteration 275 Number of M Shapes:4
Iteration 276 Number of M Shapes:2
Iteration 277 Number of M Shapes:4
Iteration 278 Number of M Shapes:2
Iteration 279 Number of M Shapes:4
Iteration 280 Number of M Shapes:4
Iteration 281 Number of M Shapes:1
Iteration 282 Number of M Shapes:2
Iteration 283 Number of M Shapes:1
Iteration 284 Number of M Shapes:4
Iteration 285 Number of M Shapes:5
Iteration 286 Number of M Shapes:2
Iteration 287 Number of M Shapes:1
Iteration 288 Number of M Shapes:4
Iteration 289 Number of M Shapes:3
Iteration 290 Number of M Shapes:4
Iteration 291 Number of M Shapes:3
Iteration 292 Number of M Shapes:3
Iteration 293 Number of M Shapes:2
Iteration 294 Number of M Shapes:4
Iteration 295 Number of M Shapes:4
Iteration 296 Number of M Shapes:1
Iteration 297 Number of M Shapes:5
Iteration 298 Number of M Shapes:2
Iteration 299 Number of M Shapes:5
Iteration 300 Number of M Shapes:2
Iteration 301 Number of M Shapes:2
Iteration 302 Number of M Shapes:1
Iteration 303 Number

<ipython-input-14-467e4b97665c>:158: RuntimeWarning: invalid value encountered in scalar divide
  if math.acos(round(np.dot((P - T), h) / TP, 6)) <= alpha:


Iteration 870 Number of M Shapes:2
Iteration 871 Number of M Shapes:3
Iteration 872 Number of M Shapes:2
Iteration 873 Number of M Shapes:1
Iteration 874 Number of M Shapes:1
Iteration 875 Number of M Shapes:4
Iteration 876 Number of M Shapes:1
Iteration 877 Number of M Shapes:4
Iteration 878 Number of M Shapes:2
Iteration 879 Number of M Shapes:2
Iteration 880 Number of M Shapes:3
Iteration 881 Number of M Shapes:1
Iteration 882 Number of M Shapes:1
Iteration 883 Number of M Shapes:1
Iteration 884 Number of M Shapes:1
Iteration 885 Number of M Shapes:1
Iteration 886 Number of M Shapes:1
Iteration 887 Number of M Shapes:2
Iteration 888 Number of M Shapes:4
Iteration 889 Number of M Shapes:2
Iteration 890 Number of M Shapes:1
Iteration 891 Number of M Shapes:4
Iteration 892 Number of M Shapes:1
Iteration 893 Number of M Shapes:2
Iteration 894 Number of M Shapes:2
Iteration 895 Number of M Shapes:2
Iteration 896 Number of M Shapes:5
Iteration 897 Number of M Shapes:4
Iteration 898 Number